In [1]:
from agential.cog.prompts.critic import (
    HUMANEVAL_FEWSHOT_EXAMPLES_POT, 
    CRITIC_POT_INSTRUCTION_HUMANEVAL,
    CRITIC_POT_INSTRUCTION_TEST_HUMANEVAL,
    HUMANEVAL_FEWSHOT_EXAMPLES_POT_TEST,
    CRITIC_CRITIQUE_INSTRUCTION_HUMANEVAL,
    HUMANEVAL_FEWSHOT_EXAMPLES_CRITIC
)
from agential.cog.functional.critic import _prompt_agent, safe_execute, _prompt_critique

import dotenv
import warnings
warnings.filterwarnings("ignore")

from langchain_community.chat_models.openai import ChatOpenAI

llm = ChatOpenAI()

In [2]:
question = """from typing import List 

def has_close_elements(numbers: List[float], threshold: float) -> bool: 
    \"\"\"Check if in given list of numbers, are any two numbers closer to each other than given threshold. 
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5) False 
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) True
    \"\"\"
"""

code = _prompt_agent(
    llm=llm,
    question=question,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_POT,
    prompt=CRITIC_POT_INSTRUCTION_HUMANEVAL,
)

<PROMPT AGENT===========================================================================>
You are an AI that only responds with python code, NOT ENGLISH. You will be given a function signature and its docstring by the user. Write your full implementation (restate the function signature, all imports, and docstring).

[function signature]:
def has_duplicate_names(names_list: List[str]) -> bool:
    """Check if there is any name that appears more than once in the list.
    >>> has_duplicate_names(['Alice', 'Bob', 'Charlie', 'Alice'])
    True
    >>> has_duplicate_names(['Alice', 'Bob', 'Charlie', 'Dave']) 
    False
    """
    return len(names_list) != len(set(names_list))

[function signature]:
def average_positive(numbers: List[int]) -> float:
    """Calculate the average of positive numbers in the list.
    >>> average_positive([1, -1, 2, -2, 3])
    2.0
    >>> average_positive([-5, 0, 5, 15])
    10.0
    >>> average_positive([100, 200, -100, 0])
    150.0
    >>> average_positive(

In [3]:
print(code)

from typing import List 

def has_close_elements(numbers: List[float], threshold: float) -> bool: 
    """Check if in given list of numbers, are any two numbers closer to each other than given threshold. 
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5) False 
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) True
    """
    for i in range(len(numbers)):
        for j in range(i + 1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False


In [4]:
tests = _prompt_agent(
    llm=llm,
    question=question,
    examples=HUMANEVAL_FEWSHOT_EXAMPLES_POT_TEST,
    prompt=CRITIC_POT_INSTRUCTION_TEST_HUMANEVAL
)

<PROMPT AGENT===========================================================================>
You are an AI coding assistant that can write unique, diverse, and intuitive unit tests for functions given the signature and docstring.

[function signature]:
def has_duplicate_names(names_list: List[str]) -> bool:
    """Check if there is any name that appears more than once in the list.
    >>> has_duplicate_names(['Alice', 'Bob', 'Charlie', 'Alice'])
    True
    >>> has_duplicate_names(['Alice', 'Bob', 'Charlie', 'Dave']) 
    False
    """

[unit tests]:
assert has_duplicate_names(['Alice', 'Bob', 'Charlie', 'Alice']) == True, "Test failed: has_duplicate_names(['Alice', 'Bob', 'Charlie', 'Alice']) should return True"
assert has_duplicate_names(['Alice', 'Bob', 'Charlie', 'Dave']) == False, "Test failed: has_duplicate_names(['Alice', 'Bob', 'Charlie', 'Dave']) should return False"

[function signature]:
def average_positive(numbers: List[int]) -> float:
    """Calculate the average of positiv

In [5]:
print(tests) 

assert has_close_elements([1.0, 2.0, 3.0], 0.5) == False, "Test failed: has_close_elements([1.0, 2.0, 3.0], 0.5) should return False"
assert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True, "Test failed: has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) should return True"


In [6]:
code_answer, execution_status = safe_execute(
    code + "\n\n" + tests #+ "assert False" + "\n\n"
)  # Can be None, "Exception".

In [7]:
execution_status

'Done'

In [11]:
a = """def exceeds_threshold(measurements: List[float], threshold: float) -> int:
    count = 0
    for i in range(1, len(measurements)):
        if abs(measurements[i] - measurements[i - 1]) > (threshold + 1):
            count += 1
    return count

assert exceeds_threshold([100, 102, 107, 103], 5) == 1, "Test failed: exceeds_threshold([100, 102, 107, 103], 5) should return 1"
assert exceeds_threshold([100, 101, 102, 103], 2) == 0, "Test failed: exceeds_threshold([100, 101, 102, 103], 2) should return 0"
"""

safe_execute(a)

(None,
 "AssertionError('Test failed: exceeds_threshold([100, 102, 107, 103], 5) should return 1')")

In [ ]:
critique_additional_keys = {
    "execution_status": execution_status,
    "code_answer": code_answer,
}

critique = _prompt_critique(
    llm=llm,
    question=code,
    examples=critique_examples,
    answer=tests,
    critique="",
    additional_keys=critique_additional_keys,
    prompt=critique_prompt
)